In [1]:
import pandas as pd
import numpy as np
import app_functions as af

In [2]:
data_dict = af.filter_data(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\pivoted_data')[0]
weights_df = pd.DataFrame(dict(Factor = data_dict.keys(),Weight = [0,0,0,0,0.2,0.3,0,0,0.4,0,0,0,0.1]))
n_quantiles = 10

2023-03-01 18:18:30.437 
  command:

    streamlit run c:\Users\hugo.perezdealbeniz\AppData\Local\miniconda3\envs\refinitiv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
weights_df = weights_df.loc[weights_df.Weight > 0]

In [4]:
ranked_data_dict = {}
columns_list = []
for factor in weights_df.Factor:
    ranked_data_dict[factor] = af.rank_data(data_dict[factor],n_quantiles)
    columns_list.append(set(ranked_data_dict[factor].columns))

In [5]:
common_columns = columns_list[0]
for i in columns_list[1:]:
    common_columns = common_columns & i
common_columns = list(common_columns)

In [6]:
weighted_df_dict = {}
for factor,ranked_df in ranked_data_dict.items():
    weight = float(weights_df.loc[weights_df.Factor==factor].Weight)
    weighted_df_dict[factor] = ranked_df[common_columns]*weight

In [7]:
dates = weighted_df_dict[list(weighted_df_dict.keys())[0]].index
final_df_list = []
for date in dates:
    summing_list = []
    try:
        for factor_df in weighted_df_dict.values():
            summing_list.append(factor_df.loc[date])
    except KeyError:
        continue
    final_df_list.append(sum(summing_list)/len(summing_list))

In [8]:
df = pd.DataFrame(final_df_list)

In [9]:
ranked_df = af.rank_data(df,n_quantiles)

In [10]:
rents_df = af.get_rents_df(ranked_df,r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\Final Data\PriceClose.csv',n_quantiles)

In [11]:
colors = ["#0068c9","#d7dce6","#7f51b5","#ffd578","#ff902d","#8af0aa","#2db19f","#ffb1b1","#ff3030","#83c9ff",'#000000']

af.plot_NAV_absoluto(rents_df,colors)